In [0]:
z.jup()

Download ipynb

In [0]:
# Lựa chọn n_layer tối ưu. Dựa trên dung lượng VRAM đang trống. 
# Với Gemma 2 9b, Mỗi layer chiếm khoảng 240MB 

# Xóa llm 
if 'llm' in locals(): del llm
if 'gemma_2_model' in locals(): del gemma_2_model

# Lấy VRAM còn trống 
_, free_GPUs = z.showUsage()
print(f'Free VRAM: {free_GPUs[0]}MB')

n_layer = int(free_GPUs[0] / 240)

if n_layer == 0:
    print(f'Sufficient VRAM (n_layer=={n_layer}), use CPU')
    n_layer = None
else:
    if n_layer > 43:
        n_layer = 43
    print(f'Optimize n_layer: {n_layer}')

---CPU---
#0: 3.2%	#1: 0.0%	#2: 3.1%
#3: 0.0%	#4: 1.6%	#5: 0.0%
#6: 3.1%	#7: 0.0%	#8: 0.0%
#9: 0.0%	#10: 1.6%	#11: 0.0%
#12: 1.6%	#13: 0.0%	#14: 0.0%
#15: 0.0%	#16: 0.0%	#17: 0.0%
#18: 0.0%	#19: 0.0%

---Memory---
29252MiB (44%) free, 65237MiB total

---GPU---
#0. Tesla P100-PCIE-16GB
memory: 5323MiB (32%) free, 16287MiB total
Free VRAM: 5323MB
Optimize n_layer: 22


In [0]:
import os
import json
import uvicorn
import asyncio
import multiprocessing

from uvicorn import Config, Server
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import JSONResponse, StreamingResponse, FileResponse

from langchain_core.documents.base import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

from langchain_community.llms import LlamaCpp
from langchain_community.chat_models import ChatLlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

from langchain_community.embeddings import LlamaCppEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_qdrant import Qdrant
from qdrant_client import QdrantClient

from tempfile import NamedTemporaryFile
from httpx import TimeoutException
from tenacity import retry, stop_after_attempt, wait_exponential
from werkzeug.datastructures import FileStorage
from fpdf import FPDF

GEMMA2_MODEL = "soc-models/gemma-2-9b-it-Q6_K_L.gguf"
EMBEDDINGS_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
# EMBEDDINGS_MODEL = "dunzhang/stella_en_1.5B_v5"

model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=EMBEDDINGS_MODEL,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
gemma_2_model = ChatLlamaCpp(
    model_path=GEMMA2_MODEL,
    verbose=False, 
    temperature=0,
    n_ctx=8192,  
    max_tokens=1024,  
    f16_kv=False,  
    n_gpu_layers=n_layer,  
    n_threads=multiprocessing.cpu_count()-1,
    streaming=True
)

app = FastAPI()

C:\Users\zeppelin\bin\python\.venv\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_path" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
C:\Users\zeppelin\bin\python\.venv\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
C:\Users\zeppelin\bin\python\.venv\Lib\site-packages\transformers\utils\hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [0]:
#---Start---llms response----

def process_template(template: str) -> str:
    template = template.replace("{", "{{")
    template = template.replace("}", "}}")
    return template

def create_prompt(template: str) -> PromptTemplate:
    processed_template = process_template(template)
    
    return PromptTemplate(
        template=processed_template,
        input_variables=[]
    )

def get_chain(template: str):
    prompt = create_prompt(template)
    
    chain = (
        prompt 
        | gemma_2_model 
        | StrOutputParser()
    )
    
    return chain
    
async def generate_result(template: str) -> str:
    chain = get_chain(template)
    response = await chain.ainvoke({})
    return response

async def generate_stream_json(template: str) -> str:
    chain = get_chain(template)
    
    async for chunk in chain.astream({}):
        yield json.dumps({
            "response": chunk,
            "success": True
        }) + "\n"

async def generate_stream_str(template: str) -> str:
    chain = get_chain(template)
    
    async for chunk in chain.astream({}):
        yield chunk

#---End---llms response----

In [0]:
#---Start---query----

DASHBOARD = 
QDRANT_URL = 
QDRANT_API_KEY = 

# COLLECTION_NAME = "DATA-SOC"
COLLECTION_NAME = "DATA-SOC-SEMANTIC"

CONTENT_PAYLOAD_KEY = "content"
METADATA_PAYLOAD_KEY = "metadata"

SAVE_PATH = './soc-data-store'
BATCH_SIZE_UPLOAD = 10

TOP_K = 3
MAX_SAME_QUERY = 1
MAX_DOCS_FOR_CONTEXT = 5

def collection_exists(client: QdrantClient, collection_name: str) -> bool:
    """Check if a Qdrant collection exists"""
    collections = client.get_collections().collections
    return any(col.name == collection_name for col in collections)

def existing_collection(collection_name: str) -> Qdrant:
    """Create vector retriever"""
    
    client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
    if not collection_exists(client, collection_name):
        return None

    doc_store = Qdrant.from_existing_collection(
        url=QDRANT_URL,
        api_key=QDRANT_API_KEY,
        embedding=embeddings,
        collection_name=collection_name,    
        content_payload_key=CONTENT_PAYLOAD_KEY,
        metadata_payload_key=METADATA_PAYLOAD_KEY
    )
    return doc_store

def similarity_search(para: dict) -> list[Document]:
    """RRF retriever"""
    common_doc_store = existing_collection(COLLECTION_NAME)
    user_doc_store = existing_collection(para["user_id"])
    
    all_results = []
    if common_doc_store:
        common_results = common_doc_store.similarity_search_with_score(para["user_query"], k=MAX_DOCS_FOR_CONTEXT)
        all_results.extend(common_results)
        
    if user_doc_store:
        user_results = user_doc_store.similarity_search_with_score(para["user_query"], k=MAX_DOCS_FOR_CONTEXT)
        all_results.extend(user_results)
    
    return all_results

async def get_ask_template(user_id: str, user_query: str) -> str:
    ssearch = RunnableLambda(similarity_search)
    context = await ssearch.ainvoke({"user_id": user_id, "user_query": user_query})
    context = [c[0].page_content for c in context]
    question = user_query
    
    template = f"""
        You are a support chatbot providing information about information security and network security.
        Please answer the following question based on the information provided and retrieve the relevant links, names of images, tables, pages, sources of the content contained in the information:

        Information: {context}
        Question: {question}
        The final answer must be translated into Vietnamese with structure:
            <no-content>
            #heading1: <content>
                <no-content>
                ##heading1.1: <content>
                    <no-content>
                    ###heading1.1.1: <content>
                        <no-content>
                        ####heading1.1.1.1: <content>
                            <no-content>
                        <no-content>
                    <no-content>
                    ###heading1.1.2: <content>
                        <no-content>
                    <no-content>
                <no-content>
            <no-content>
            #heading2: <content>
                <no-content>
                #heading2.1: <content>
                    <no-content>
                <no-content>
            <no-content>
            #heading3: <content>
                <no-content>
                ##heading3.1: <content>
                    <no-content>
                <no-content>
                ##heading3.2: <content>
                    <no-content>
                <no-content>
            <no-content>
            **images**: https://example.com/images1, https://example.com/images2, ...
            **tables**: https://example.com/tables1, https://example.com/tables2, ...
            **pages**: https://example.com/pages1, https://example.com/pages2, ...
            **sources**: https://example.com/sources1, https://example.com/sources2, ...
        
        Question example: Xác thực 2 yếu tố là gì?
        Output example: 
            #heading1: Xác thực hai yếu tố (2FA) là một biện pháp bảo mật bổ sung yêu cầu người dùng cung cấp hai hình thức xác minh khác nhau để đăng nhập vào tài khoản của họ.
            #heading2: Các loại xác thực 2FA
                ##heading2.1: Yếu tố đầu tiên: Thông thường là tên người dùng và mật khẩu.
                ##heading2.2: Yếu tố thứ hai: Có thể bao gồm:
                    ###heading2.2.1: Mã OTP (One-Time Password) được gửi qua SMS hoặc email.
                    ###heading2.2.2: Ứng dụng xác thực trên điện thoại di động (ví dụ: Google Authenticator, Authy).
                    ###heading2.2.3: Dòng chữ ký vật lý (FIDO2).
                    ###heading2.2.4: Chuyển đổi sinh trắc học (nhận dạng khuôn mặt, vân tay).
            #heading3: Lợi ích của Xác thực 2FA
                ##heading3.1: Xác thực 2FA giúp tăng cường bảo mật tài khoản bằng cách làm cho việc truy cập trái phép trở nên khó khăn hơn đáng kể. Bằng cách yêu cầu hai yếu tố xác minh, 2FA ngăn chặn kẻ tấn công có được quyền truy cập vào tài khoản của bạn ngay cả khi họ biết tên người dùng vàmật khẩu của bạn.
            **sources**: https://www.idrbt.ac.in/wp-content/uploads/2022/07/ISOC.pdf
    """
    return template

#---End---Query----

In [0]:
#---Start---evaluate----

n_TODO = 31
TODO_LIST = [
    """**Review and update information security policies**""",
    """**Clearly define the policy change approval process**""",
    """**Assess risks from third-party service providers**""",
    """**Establish agreements with third-party service providers**""",
    """**Set up a formal risk management program**""",
    """**Develop an ethics violation handling process**""",
    """**Review and update access management procedures**""",
    """**Provide details on logical access controls**""",
    """**Enhance physical access controls**""",
    """**Upgrade power backup systems**""",
    """**Conduct regular penetration testing**""",
    """**Provide details on incident management, incident response, and change management procedures**""",
    """**Configure disaster recovery for production servers**""",
    """**Improve patch management procedures**""",
    """**Enhance security monitoring**""",
    """**Improve performance monitoring**""",
    """**Enhance the ticketing system**""",
    """**Perform more frequent vulnerability scanning**""",
    """**Improve intrusion detection and prevention**""",
    """**Configure and manage firewalls**""",
    """**Provide details on backup and data recovery procedures**""",
    """**Enhance recovery testing**""",
    """**Establish an IT asset management process**""",
    """**Improve vendor management processes**""",
    """**Enhance employee security awareness training**""",
    """**Provide security training for customers**""",
    """**Improve recruitment, training, and termination processes**""",
    """**Perform operational effectiveness testing**""",
    """**Consider SOC 2 Type 2 audit**""",
    """**Ensure compliance with security regulations and standards**""",
    """**Conduct regular security assessments**)"""
]

TODO_LIST_STR = "\n".join([f"{i + 1}. {item}" for i, item in enumerate(TODO_LIST)])

def process_result_of_evaluate(result: str) -> list[int]:
    newresult = ''.join((ch if ch in '0123456789.-e' else ' ') for ch in result)
    listOfNumbers = [int(i) for i in newresult.split() if i.isdigit() and int(i) > 0 and int(i) <= n_TODO]
    return listOfNumbers

def get_evaluate_template(todo_list_str: str, content: str) -> dict:
    """Query with vector db"""
    
    template = f"""
        Please evaluate the security-related content based on the "content" provided, which definition and level of requirements are satisfied in the "To-do List":

        To-do List: {todo_list_str}
        Content: {content}
        The final answer is the positions in the to-do list, only the positions are given, if there is no result or cannot be evaluated or the result is ambiguous and unclear during the evaluation, the answer must be: -1
        Output example 1: 1, 2, 3
        Output example 2: 3, 7, 12, 23
        Output example 3: -1 
    """

    return template

#---End---evaluate----

In [0]:
#---Start---save file----

def check_file_type(file_path: str):
    _, file_extension = os.path.splitext(file_path)
    if file_extension.lower() == ".pdf":
        return "PDF"
    elif file_extension.lower() == ".docx":
        return "DOCX"
    else:
        return "Unknown"

def get_file_name(file_path: str):
    file_name = file_path.split('\\')[-1]
    folder_path = file_path.split('\\')[:-1]
    return file_name

def get_folder_abs_path(user_id: str) -> str:
    if not os.path.exists(SAVE_PATH):
        os.makedirs(SAVE_PATH)

    folder = SAVE_PATH + "/" + user_id
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    folder_abs_path = os.path.abspath(folder)
    return folder_abs_path

async def save_pdf(file: str, user_id: str) -> str:
    folder_abs_path = get_folder_abs_path(user_id)
    
    pdf_content = await file.read()
    file_name = file.filename
    file_abs_path = os.path.join(folder_abs_path, file_name)

    with open(file_abs_path, "wb") as output_file:
        output_file.write(pdf_content)
    
    return file_abs_path
    
def get_check_is_socreport_template(sample_content: str):
    template = f"""
        Are you a chatbot that helps classify whether the provided "content sample" is part of the "SOC report" content?
        
        Content sample: {sample_content}
        
        **No additional content is generated.**
        **The final answer is True or False.**
        **If there is no result or cannot be evaluated or the result is unclear and ambiguous during the evaluation, the answer must be: False**
        
        Example output 1: True
        Example output 2: False
        Example output 3: True
    """
    return template
    
async def check_is_socreport_file(sample_content: str):
    template = get_check_is_socreport_template(sample_content)
    result = await generate_result(template)
    result = result.lower().strip()
    print(result)
    
    if result.lower() == "true": return True
    return False
    
    
#---End---save file----

In [0]:
#---Start---create file PDF----

def initialize_pdf():
    pdf = FPDF('P', 'mm', 'A4')
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_font("DejaVu", "", "soc-fonts/DejaVuSans.ttf", uni=True)
    pdf.add_font("DejaVu-Bold", "", "soc-fonts/DejaVuSans-Bold.ttf", uni=True)
    pdf.add_font("DejaVu-Italic", "", "soc-fonts/DejaVuSerif-Italic.ttf", uni=True)
    return pdf

def add_header(pdf, title:str):
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, title, 0, 1, 'C')

def add_section_title(pdf, title:str):
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(0, 10, title, 0, 1)

def add_task_list(pdf, todo_list:list[str], indices:dict, title:str):
    add_section_title(pdf, title)
    pdf.set_font("DejaVu", '', 13)
    line_height = 10
    
    for i, (key, value) in enumerate(indices.items()):
        if key < 0 or key >= len(todo_list): continue
    
        checkbox = "\u2611" if value else "\u2610"
        title_number = "1" if value else "2"
        parts = todo_list[key].split("**")
        
        pdf.set_x(pdf.get_x() + 10)
        pdf.set_font("DejaVu-Bold", '', 13)
        pdf.multi_cell(0, line_height, f"{title_number}.{i+1}. {checkbox} {parts[1].strip()}" , 0)

        # pdf.set_x(pdf.get_x() + 10)
        # pdf.set_font("DejaVu", '', 13)
        # pdf.multi_cell(0, line_height, f"\u279B{parts[2].strip()}", 0)
        
        # if value:
        #     pdf.set_x(pdf.get_x() + 10)
        #     pdf.set_font("DejaVu-Italic", '', 13)
        #     pdf.multi_cell(0, line_height, f"\u201C\u2026{value.page_content.strip()}\u2026\u201D", 0)
            
        pdf.ln(5)

def create_information_security_evaluation(todo_list:list[str], completed_indices:dict, user_id:str, file_evaluation_name:str="Information_Security_Evaluation.pdf") -> str:
    folder_abs_path = get_folder_abs_path(user_id)
    output_path = os.path.join(folder_abs_path, file_evaluation_name)
    
    pdf = initialize_pdf()
    add_header(pdf, "Information Security Evaluation")
    
    add_task_list(pdf, todo_list, completed_indices, "1. Completed Tasks",)
    pdf.add_page()
    
    uncompleted_positions = set(range(len(todo_list))) - set(completed_indices.keys())
    uncompleted_positions = {pos: None for pos in uncompleted_positions}
    add_task_list(pdf , todo_list, uncompleted_positions, "2. Pending Tasks")
    
    pdf.output(output_path)
    return file_evaluation_name, output_path

#---End---create file PDF----

In [0]:
#---Start---document----

def get_raw_documents(file_path: str) -> list[Document]:
    file_type = check_file_type(file_path)
    if file_type == "PDF":
        raw_documents = PyPDFLoader(file_path).load()
    elif file_type == "DOCX":
        raw_documents = Docx2txtLoader(file_path).load()
    else:
        raw_documents = []
    return raw_documents
    
def process_raw_documents(raw_documents: list[Document]) -> list[Document]: 
    source = get_file_name(raw_documents[0].metadata.get('source'))
    for idx, raw_document in enumerate(raw_documents):
        page = raw_document.metadata.get('page')
        metadata = {'source': source, 'page': page}
        setattr(raw_document, 'metadata', metadata)
    return raw_documents
    
def get_documents(file_path: str) -> list[Document]:
    raw_documents = get_raw_documents(file_path)
    documents = process_raw_documents(raw_documents)
    return documents
    
def get_texts(file_path: str) -> (str, str):
    raw_documents = get_raw_documents(file_path)
    source = raw_documents[0].metadata.get('source').split('\\')[-1]
    page_content = ""
    for raw_document in raw_documents:
        page_content += raw_document.page_content
    return source, page_content

def process_documents_before_store(documents: list[Document]) -> list[Document]:
    pre_page = 0
    for index, document in enumerate(documents):
        size = len(document.page_content)
        metadata = document.metadata
        metadata['size'] = size
        metadata['index'] = index
        metadata['pre_page'] = pre_page
        pre_page = metadata.get('page')
        setattr(document, 'metadata', metadata)
        
    return documents

def split_documents_file_common(file_path: str) -> list[Document]:
    text_splitter = SemanticChunker(
        embeddings=embeddings, 
        breakpoint_threshold_type="percentile", 
        number_of_chunks=1024
    )
    
    documents = get_documents(file_path)
    documents = text_splitter.split_documents(documents)
    documents = process_documents_before_store(documents)
    return documents
    
def split_documents_file_soc(file_path: str) -> list[Document]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 6144, 
        chunk_overlap = 124,
        separators = [
            "\n\n",
            "\n",
            " ",
            ".",
            ",",
            "\u200b",  # Zero-width space
            "\uff0c",  # Fullwidth comma
            "\u3001",  # Ideographic comma
            "\uff0e",  # Fullwidth full stop
            "\u3002",  # Ideographic full stop
            "",
        ]
    )
    
    source, page_content = get_texts(file_path)
    documents = text_splitter.create_documents([page_content])
    return documents
    
#---End---document----

In [0]:
#---Start---upload----

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def upload_to_qdrant(docs: list[Document], user_id: str):
    try:
        Qdrant.from_documents(
            documents=docs,
            embedding=embeddings,
            url=QDRANT_URL,
            api_key=QDRANT_API_KEY,
            collection_name=user_id,
            content_payload_key=CONTENT_PAYLOAD_KEY,
            metadata_payload_key=METADATA_PAYLOAD_KEY,
        )
    except TimeoutException as e:
        print(f"Timeout occurred: {e}")
        raise 

async def upload_common(file_abs_path: str, user_id: str) -> bool:
    docs = split_documents_file_common(file_abs_path)

    for i in range(0, len(docs), BATCH_SIZE_UPLOAD):
        batch = docs[i:i + BATCH_SIZE_UPLOAD]
        try:
            upload_to_qdrant(batch, user_id)
        except TimeoutException:
            print(f"Failed to upload batch {i // BATCH_SIZE_UPLOAD + 1}. Moving to the next batch.")
            return False
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            return False
    
    return True
    
async def upload_soc(file_abs_path: str, user_id: str):
    # completed_positions = {}
    # docs = split_documents_file_soc(file_abs_path)
    
    # print("Length", len(docs))
    # print("File name", file_abs_path)
    
    # for idx, doc in enumerate(docs):
    #     template = get_evaluate_template(TODO_LIST_STR, doc)
    #     print("Template size:", len(template))
        
    #     async for chunk in generate_stream_str(template):
    #         completed_position_list = process_result_of_evaluate(chunk)
            
    #         if completed_position_list == []: 
    #             continue
            
    #         for completed_position in completed_position_list:
    #             completed_positions[completed_position] = doc
        
    #     create_information_security_evaluation(TODO_LIST, completed_positions, user_id)
    #     print("Done index: ", idx)
            
    # file_evaluation_name, pdf_output_path = create_information_security_evaluation(TODO_LIST, completed_positions, user_id)
    file_evaluation_name = "Information_Security_Evaluation.pdf"
    pdf_output_path = "./soc-data-store/user1/Information_Security_Evaluation.pdf"
    # print("Completed positions", completed_positions)
    print(f"PDF output path: {pdf_output_path}")
    
    return file_evaluation_name, pdf_output_path

async def upload(file: FileStorage, user_id: str, save_file: bool = True):
    result_upload_common = None
    result_upload_soc = None
    
    file_abs_path = await save_pdf(file, user_id)
    print("1")
    result_upload_common = await upload_common(file_abs_path, user_id)
    print("2")
    
    docs = split_documents_file_common(file_abs_path)
    sample_content = docs[0]
    print(sample_content)
    
    is_socreport_file = await check_is_socreport_file(sample_content)
    print("is", is_socreport_file)
    if is_socreport_file:
        print("Hehe")
        result_upload_soc = await upload_soc(file_abs_path, user_id)
    
    if os.path.exists(file_abs_path) and not save_file: 
        print("434")
        os.remove(file_abs_path)
        
    return (result_upload_common, result_upload_soc, is_socreport_file)

#---End---upload----

In [0]:
#---Start---API----

@app.post("/llms")
async def post_llms(template: str = Form(...)):
    return StreamingResponse(
        generate_stream_json(template), media_type='text/event-stream'
    )

@app.post("/ask")
async def post_ask(user_id: str = Form(...), user_query: str = Form(...)):
    template = await get_ask_template(user_id, user_query)
    print("ask", user_id)
    print("ask", user_query)
    
    return StreamingResponse(
        generate_stream_json(template), media_type='text/event-stream'
    )

@app.post("/upload")
async def post_upload(user_id: str = Form(...), file: UploadFile = File(...)):
    print("upload-common", user_id)
    print("upload-common", file.filename)
    
    if file and user_id:
        result_upload_common, result_upload_soc, is_socreport_file = await upload(file, user_id, True)
        print("result_upload_common, result_upload_soc, is_socreport_file", result_upload_common, result_upload_soc, is_socreport_file)
        if is_socreport_file:
            file_evaluation_name, pdf_output_path = result_upload_soc
            print("file_evaluation_name, pdf_output_path", file_evaluation_name, pdf_output_path)
            return FileResponse(pdf_output_path, media_type='application/pdf', filename=file_evaluation_name)
    
    return JSONResponse(
        content={
            "response": "Tải lên file không thành công, vui lòng kiểm tra lại file pdf của bạn!",
            "success": False
        },
        status_code=200
    )

#---End---API----

In [0]:
async def main():
    config = Config(
        app=app,
        host="0.0.0.0",
        port=7722,
        log_level="debug",
        log_config=None,
        use_colors=False
    )
    
    server = Server(config)
    await server.serve()

asyncio.run(main())

java.net.SocketException: Connection reset
	at java.net.SocketInputStream.read(SocketInputStream.java:210)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at java.io.BufferedInputStream.fill(BufferedInputStream.java:246)
	at java.io.BufferedInputStream.read1(BufferedInputStream.java:286)
	at java.io.BufferedInputStream.read(BufferedInputStream.java:345)
	at org.apache.thrift.transport.TIOStreamTransport.read(TIOStreamTransport.java:127)
	at org.apache.thrift.transport.TTransport.readAll(TTransport.java:86)
	at org.apache.thrift.protocol.TBinaryProtocol.readAll(TBinaryProtocol.java:429)
	at org.apache.thrift.protocol.TBinaryProtocol.readI32(TBinaryProtocol.java:318)
	at org.apache.thrift.protocol.TBinaryProtocol.readMessageBegin(TBinaryProtocol.java:219)
	at org.apache.thrift.TServiceClient.receiveBase(TServiceClient.java:69)
	at org.apache.zeppelin.interpreter.thrift.RemoteInterpreterService$Client.recv_interpret(RemoteInterpreterService.java:274)
	at org.apache.zeppel